# Import

In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from sqlalchemy import create_engine
import pandas as pd
import json
from collections import defaultdict
import matplotlib.pyplot as plt
import datetime

In [7]:
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.style.use('fivethirtyeight')

# Calculate Status for all the patients that has responses

### Query data

In [8]:
df_patient_response = pd.read_csv('../../data/patient_response.csv')
df_patient = pd.read_csv('../../data/patient_detail.csv')
df_tmetryentry = pd.read_csv('../../data/tmetryentry.csv')

In [73]:
df_patient_response

,Unnamed: 0,id,checklist_id,question_id,workday,response,date,patient_id,unit_id,user_detail_id,hash_fin,just_date
0,24,52,1,$0101,2017-05-15,Y,2017-05-15 13:09:54.224097,54,4,69,40dab740d74ff92253b9d3c2bee5fbb26e665f26,2017-05-15
1,25,53,1,$0201,2017-05-15,N,2017-05-15 13:10:22.898225,54,4,69,40dab740d74ff92253b9d3c2bee5fbb26e665f26,2017-05-15
2,26,54,1,$0203,2017-05-15,N,2017-05-15 13:10:24.263776,54,4,69,40dab740d74ff92253b9d3c2bee5fbb26e665f26,2017-05-15
3,27,55,1,$0801,2017-05-15,Y,2017-05-15 15:03:29.340876,46,3,67,40dab740d74ff92253b9d3c2bee5fbb26e665f26,2017-05-15
4,28,56,1,$0801,2017-05-15,Y,2017-05-15 13:42:35.473971,55,4,69,40dab740d74ff92253b9d3c2bee5fbb26e665f26,2017-05-15
5,29,57,1,$0701,2017-05-15,Y,2017-05-15 13:42:42.653400,55,4,69,40dab740d74ff92253b9d3c2bee5fbb26e665f26,2017-05-15
6,30,58,1,$0703,2017-05-15,A,2017-05-15 13:42:43.545755,55,4,69,40dab740d74ff92253b9d3c2bee5fbb26e665f26,2017-05-15
7,31,59,1,$0601,2017-05-15,N,2017-05-15 13:43:22.262356,55,4,69,40dab740d74ff92253b9d3c2bee5fbb26e665f26,2017-05-15
8,32,60,1,$0301,2017-05-15,Y,2017-05-15 13:43:59.370503,55,4,69,40dab740d74ff92253b9d3c2bee5fbb26e665f26,2017-05-15
9,33,61,1,$0401,2017-05-15,A,2017-05-15 13:44:03.955529,55,4,69,40dab740d74ff92253b9d3c2bee5fbb26e665f26,2017-05-15


### How each dataframe looks like

In [9]:
#Information for all of the entries
df_tmetryentry.head()

,Unnamed: 0,id,session_id,date,object,action,strValue,unit_id,user_detail_id,intValue,just_date
0,28620,29918,2139699710,2017-05-15 13:07:14.407210,1,201,nm177724,3.0,67,NaN,2017-05-15
1,28621,29919,2139699710,2017-05-15 13:07:14.536299,1,203,nm177724,3.0,67,NaN,2017-05-15
2,28622,29920,2139699710,2017-05-15 13:07:14.536680,2,204,nm177724,3.0,67,NaN,2017-05-15
3,28623,29921,2139699710,2017-05-15 13:07:14.567145,2,301,nm177724,3.0,67,NaN,2017-05-15
4,28624,29922,2139699710,2017-05-15 13:07:14.567565,3,302,nm177724,3.0,67,NaN,2017-05-15


In [10]:
df_tmetryentry[df_tmetryentry.strValue == '098d477fde14ee3ee30c9f363d5ac06f85c3ca51']

,Unnamed: 0,id,session_id,date,object,action,strValue,unit_id,user_detail_id,intValue,just_date
23,28643,29941,2139699710,2017-05-15 13:09:37.229240,6,602,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,3.0,67,NaN,2017-05-15
60,28680,29978,2139699710,2017-05-15 13:12:51.209708,6,602,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,3.0,67,NaN,2017-05-15
70,28690,29988,2139699710,2017-05-15 13:16:12.955344,6,602,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,3.0,67,NaN,2017-05-15
107,28775,30073,3547602005,2017-05-15 13:20:51.575758,6,602,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,3.0,67,NaN,2017-05-15
212,29154,30452,172184393,2017-05-15 14:33:32.844021,6,602,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,3.0,67,NaN,2017-05-15
395,29389,30687,334320999,2017-05-16 14:47:00.542595,6,602,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,3.0,67,NaN,2017-05-16
660,251968,253266,3860602040,2017-05-17 14:46:13.181498,6,602,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,3.0,67,NaN,2017-05-17


In [11]:
#information on the answers
df_patient_response[df_patient_response.patient_id.isin([120])]

,Unnamed: 0,id,checklist_id,question_id,workday,response,date,patient_id,unit_id,user_detail_id,hash_fin,just_date
736,1741,1783,2,$0401,2017-05-31,A,2017-05-31 14:42:59.749305,120,4,71,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,2017-05-31
737,1742,1784,2,$0402,2017-05-31,Y,2017-05-31 14:43:00.902044,120,4,71,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,2017-05-31
738,1743,1785,2,$0501,2017-05-31,N,2017-05-31 14:43:02.948959,120,4,71,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,2017-05-31
739,1744,1786,2,$0701,2017-05-31,N,2017-05-31 14:43:06.083190,120,4,71,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,2017-05-31
740,1745,1787,2,$0702,2017-05-31,Y,2017-05-31 14:43:07.549812,120,4,71,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,2017-05-31
741,1746,1788,2,$0703,2017-05-31,N,2017-05-31 14:43:08.783249,120,4,71,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,2017-05-31
742,1747,1789,2,$0801,2017-05-31,Y,2017-05-31 14:43:11.433846,120,4,71,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,2017-05-31
743,1748,1790,2,$0301,2017-05-31,N,2017-05-31 14:43:39.367822,120,4,71,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,2017-05-31
829,1846,1888,2,$0801,2017-06-01,Y,2017-06-01 14:32:14.329502,120,4,75,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,2017-06-01
830,1847,1889,2,$0401,2017-06-01,N,2017-06-01 14:35:19.665822,120,4,75,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,2017-06-01


In [12]:
#information on the patients
df_patient.head()

,Unnamed: 0,id,hash_fin,date,checklist_id,visible_groups,los_icu,los,apache_is_completed,has_antibiotics,has_lines,has_medication,has_ventilation,workday,unit_id,user_detail_id,just_date
0,0,44,99b2749dd914e4a98ae987f37356a7df60c840c1,2017-04-26 19:40:32.087904,1,"08,07,03",3 days,3 days,1,0,1,1,0,2017-04-26,3,7,2017-04-26
1,1,45,99b2749dd914e4a98ae987f37356a7df60c840c1,2017-04-26 19:42:33.481153,1,"07,03,08",3 days,3 days,1,0,1,1,0,2017-04-26,3,7,2017-04-26
2,2,46,40dab740d74ff92253b9d3c2bee5fbb26e665f26,2017-04-28 19:40:00.586307,1,"06,08,07,03",1 hour,2 days,1,1,1,1,0,2017-04-28,3,7,2017-04-28
3,3,47,517f5c02fee6304bf8f97ed13a9ce8399f44456b,2017-04-28 19:41:07.897426,1,"08,07,06,03",1 day,1 day,1,1,1,1,0,2017-04-28,3,7,2017-04-28
4,4,48,40dab740d74ff92253b9d3c2bee5fbb26e665f26,2017-04-28 19:41:26.418122,1,"03,08,07,06",1 hour,2 days,1,1,1,1,0,2017-04-28,3,7,2017-04-28


In [13]:
df_patient[df_patient.hash_fin == '07e1b75fe53343221703226deb96b71bb6d90930']

,Unnamed: 0,id,hash_fin,date,checklist_id,visible_groups,los_icu,los,apache_is_completed,has_antibiotics,has_lines,has_medication,has_ventilation,workday,unit_id,user_detail_id,just_date
19,19,95,07e1b75fe53343221703226deb96b71bb6d90930,2017-05-04 18:34:07.824293,1,"01,07,08,06,03",7 days,15 days,1,1,1,1,1,2017-05-04,4,7,2017-05-04
64,64,140,07e1b75fe53343221703226deb96b71bb6d90930,2017-05-15 15:37:12.688794,1,"01,06,03,08,07",18 days,26 days,1,1,1,1,1,2017-05-15,4,69,2017-05-15
70,70,146,07e1b75fe53343221703226deb96b71bb6d90930,2017-05-16 14:57:27.915578,1,"01,06,08,03,07",19 days,27 days,1,1,1,1,1,2017-05-16,4,69,2017-05-16
87,87,163,07e1b75fe53343221703226deb96b71bb6d90930,2017-05-17 14:42:12.291822,1,"01,07,06,08,03",20 days,28 days,1,1,1,1,1,2017-05-17,4,69,2017-05-17
131,131,207,07e1b75fe53343221703226deb96b71bb6d90930,2017-05-18 15:07:32.821076,1,"01,07,06,03,08",21 days,29 days,1,1,1,1,1,2017-05-18,4,72,2017-05-18
153,153,229,07e1b75fe53343221703226deb96b71bb6d90930,2017-05-19 15:17:01.053216,1,"01,03,08,07,06",22 days,30 days,1,1,1,1,1,2017-05-19,4,71,2017-05-19
154,154,230,07e1b75fe53343221703226deb96b71bb6d90930,2017-05-19 15:19:09.764039,1,"01,03,08,07,06",22 days,30 days,1,1,1,1,1,2017-05-19,4,71,2017-05-19


In [14]:
print(df_patient_response.columns, df_patient.columns)

Index(['Unnamed: 0', 'id', 'checklist_id', 'question_id', 'workday',
       'response', 'date', 'patient_id', 'unit_id', 'user_detail_id',
       'hash_fin', 'just_date'],
      dtype='object') Index(['Unnamed: 0', 'id', 'hash_fin', 'date', 'checklist_id',
       'visible_groups', 'los_icu', 'los', 'apache_is_completed',
       'has_antibiotics', 'has_lines', 'has_medication', 'has_ventilation',
       'workday', 'unit_id', 'user_detail_id', 'just_date'],
      dtype='object')


In [15]:
#matching patients with the answers
df1 = pd.merge(df_patient_response, df_patient[['id', 'hash_fin', 'date', 'checklist_id', 'unit_id',
                                                'visible_groups', 'los_icu', 'user_detail_id', 'just_date']],            
              
               left_on=['hash_fin','user_detail_id','unit_id','checklist_id'], 
               right_on=['hash_fin', 'user_detail_id','unit_id','checklist_id'])
# df_merge_patient_response[['checklist_id_x', 'question_id_x', 'workday_x', 'response', 'date_x', 'patient_id',]]
# df_merge_patient_response.columns

In [74]:
#Please rename this, g1 has all the responses for each patient 
g = df1[df1.date_x >= df1.date_y].groupby(['hash_fin', 'id_x', 'workday'],as_index=False)#user_detail_id', 'unit_id', 'checklist_id','workday'])#['date_x', 'visible_groups']
g1 = g.apply(lambda x: x.sort_values(by='date_y', ascending=False))
g1.drop_duplicates(subset='id_x', keep='first')
g1

,,Unnamed: 0,id_x,checklist_id,question_id,workday,response,date_x,patient_id,unit_id,user_detail_id,hash_fin,just_date_x,id_y,date_y,visible_groups,los_icu,just_date_y
0,87,935,977,1,$0101,2017-05-23,Y,2017-05-23 14:54:07.267612,95,4,72,07e1b75fe53343221703226deb96b71bb6d90930,2017-05-23,207,2017-05-18 15:07:32.821076,"01,07,06,03,08",21 days,2017-05-18
1,88,936,978,1,$0201,2017-05-23,N,2017-05-23 14:54:08.838137,95,4,72,07e1b75fe53343221703226deb96b71bb6d90930,2017-05-23,207,2017-05-18 15:07:32.821076,"01,07,06,03,08",21 days,2017-05-18
2,89,937,979,1,$0203,2017-05-23,N,2017-05-23 14:54:11.885950,95,4,72,07e1b75fe53343221703226deb96b71bb6d90930,2017-05-23,207,2017-05-18 15:07:32.821076,"01,07,06,03,08",21 days,2017-05-18
3,90,938,980,1,$0601,2017-05-23,N,2017-05-23 14:54:14.358476,95,4,72,07e1b75fe53343221703226deb96b71bb6d90930,2017-05-23,207,2017-05-18 15:07:32.821076,"01,07,06,03,08",21 days,2017-05-18
4,91,939,981,1,$0701,2017-05-23,Y,2017-05-23 14:54:25.799800,95,4,72,07e1b75fe53343221703226deb96b71bb6d90930,2017-05-23,207,2017-05-18 15:07:32.821076,"01,07,06,03,08",21 days,2017-05-18
5,92,940,982,1,$0702,2017-05-23,Y,2017-05-23 14:54:20.571559,95,4,72,07e1b75fe53343221703226deb96b71bb6d90930,2017-05-23,207,2017-05-18 15:07:32.821076,"01,07,06,03,08",21 days,2017-05-18
6,93,941,983,1,$0703,2017-05-23,Y,2017-05-23 14:54:28.054966,95,4,72,07e1b75fe53343221703226deb96b71bb6d90930,2017-05-23,207,2017-05-18 15:07:32.821076,"01,07,06,03,08",21 days,2017-05-18
7,94,942,984,1,$0801,2017-05-23,Y,2017-05-23 14:54:30.321690,95,4,72,07e1b75fe53343221703226deb96b71bb6d90930,2017-05-23,207,2017-05-18 15:07:32.821076,"01,07,06,03,08",21 days,2017-05-18
8,95,943,985,1,$0301,2017-05-23,N,2017-05-23 14:54:32.821528,95,4,72,07e1b75fe53343221703226deb96b71bb6d90930,2017-05-23,207,2017-05-18 15:07:32.821076,"01,07,06,03,08",21 days,2017-05-18
9,96,944,986,1,$0401,2017-05-23,N,2017-05-23 14:54:33.801111,95,4,72,07e1b75fe53343221703226deb96b71bb6d90930,2017-05-23,207,2017-05-18 15:07:32.821076,"01,07,06,03,08",21 days,2017-05-18


# Specific Categories

Yeah, so they want to look at three specific questions: 1) DVT, 2) Lines (specifically central lines if their version has it split into peripheral and central), 3) Foleys


[2:01] 
I think for each question, what they want to know is: for each week, 1) how many patients total were the questions answered, 2) for the patients where it was answered, what are the proportions of the different answers, 3) for patients where the 'bad' answer was given initially (no we can't discontinue the foley, no we can't pull the line, etc), how many days did they spend in the 'bad' answer category before switching to the 'good' category. [Taking into account where it was contraindicated]


[2:02] 
they want to see it because they measure all these things by another method (probably nursing reports) and they want to see if those two sources (us vs nursing reports) have any differences

In [84]:
df_tmetryentry[(pd.to_datetime(df_tmetryentry.just_date)>=cutoff_lst[4]) &(pd.to_datetime(df_tmetryentry.just_date)<cutoff_lst[5])]['strValue'].unique()

array(['cstrebin', nan, 'theeren',
       'b7a0edbb7717b8cd2235f87e6d73ea74073e400d', '08', '07', '06', '03',
       'e899dc55a23a0a8c01769c5a56f03dc4e9f17505', '01',
       '6ee5b072c8014dca95fc5b5501ba7f7f1d3514c7',
       '9d6ee4bf3b61e3656bae1f8845f7736b5eff9db3',
       '8fab61d31d1462a082e55783441fd54663904665', 'linesFood',
       'a561b67d55fb75d65ad4dabe8e0f9ed7d7749dd7', 'dvt',
       '0345b17b7ecb39019a79c591d592a4856fff5839',
       '3dd0a925afd608e7b10e9a74af260825423e3e86',
       'e75101af0ff2b46742473dc93d4408dd0a6fbc0c',
       '0263d40ce7ca75676a310008a9ce8a3d4e48b517',
       'f738cc5a32b7fde9547403f46781b4ac7b26f1c2',
       '44eb14a4e39dc68178a882726d946704a807d031',
       '9ff6cb7ff05b8cbf9342a0102bce5c43ed35e3af',
       '0246d013c22cfb04b4f78e48c69d6457549150fe',
       'ccb48b9d976321bab7ef53af92b6efca2a5c5ba6', 'antibiotics',
       'nm177725', 'jesturne', '4b26f669fb77d4a8fca62df8bab6569bd076c1ae',
       '10', 'd9c65fc5beeece19a369fbb48caeedd2d2863c9d',
   

In [75]:
g1['workday'] = pd.to_datetime(g1['workday'])

In [81]:
cutoff_lst

[Timestamp('2017-05-22 00:00:00'),
 Timestamp('2017-05-29 00:00:00'),
 Timestamp('2017-06-05 00:00:00'),
 Timestamp('2017-06-12 00:00:00'),
 Timestamp('2017-06-19 00:00:00'),
 Timestamp('2017-06-26 00:00:00'),
 Timestamp('2017-07-03 00:00:00')]

In [80]:
g1[(g1.workday>=cutoff_lst[2]) &(g1.workday<cutoff_lst[3])]['hash_fin'].unique()

array(['2265516f326cf9ea002bfd4a21dc122a63e0e380',
       '7a2306cbdeb6c8d6087279f0df15bbfdb3df0218'], dtype=object)

In [76]:
#create list of unique workdays
workday_lst = sorted(list(set(g1.workday)))

#group by week/date
#need to establish week cutoffs, first day is May 22, 2017 - Monday - Sunday
#start day
start_day = pd.Timestamp('2017-05-22 00:00:00')
cutoff_lst = [start_day]
tester = True

while tester==True:
    #add 7 days to it, check to see if it's after the last workday
    newday = cutoff_lst[-1]+pd.Timedelta(days=7)
    
    if newday >= workday_lst[-1]:
        cutoff_lst.append(newday)
        tester=False
    else:
        cutoff_lst.append(newday)


#filter the df based on the week
#go through each patient - check if visible group has it
week_df = g1[(g1.workday>=cutoff_lst[4]) &(g1.workday<cutoff_lst[5])& (g1.visible_groups.str.contains('03'))]
#need to get unique pairings of patient and workday
week_ptdt = list(set(zip(week_df.hash_fin, week_df.workday)))

print(week_ptdt)
print()

#find out how many pts were answered
answered_df = week_df[week_df.question_id=='$0401']
answered_ptdt = list(set(zip(answered_df.hash_fin, answered_df.workday, answered_df.question_id)))
print(answered_ptdt)

#if answered, what was the answer

[('7a2306cbdeb6c8d6087279f0df15bbfdb3df0218', Timestamp('2017-06-20 00:00:00')), ('7a2306cbdeb6c8d6087279f0df15bbfdb3df0218', Timestamp('2017-06-21 00:00:00')), ('f3b69dd81ffa2ed50de3166f1f9b99cbb0c6de89', Timestamp('2017-06-20 00:00:00')), ('7a2306cbdeb6c8d6087279f0df15bbfdb3df0218', Timestamp('2017-06-19 00:00:00')), ('7a2306cbdeb6c8d6087279f0df15bbfdb3df0218', Timestamp('2017-06-25 00:00:00'))]

[('f3b69dd81ffa2ed50de3166f1f9b99cbb0c6de89', Timestamp('2017-06-20 00:00:00'), '$0401'), ('7a2306cbdeb6c8d6087279f0df15bbfdb3df0218', Timestamp('2017-06-25 00:00:00'), '$0401'), ('7a2306cbdeb6c8d6087279f0df15bbfdb3df0218', Timestamp('2017-06-21 00:00:00'), '$0401'), ('7a2306cbdeb6c8d6087279f0df15bbfdb3df0218', Timestamp('2017-06-19 00:00:00'), '$0401'), ('7a2306cbdeb6c8d6087279f0df15bbfdb3df0218', Timestamp('2017-06-20 00:00:00'), '$0401')]


In [43]:
week_df

Unnamed: 0  \
hash_fin                                 id_x workday                      
07e1b75fe53343221703226deb96b71bb6d90930 977  2017-05-23 87          935   
                                         978  2017-05-23 88          936   
                                         979  2017-05-23 89          937   
                                         980  2017-05-23 90          938   
                                         981  2017-05-23 91          939   
                                         982  2017-05-23 92          940   
                                         983  2017-05-23 93          941   
                                         984  2017-05-23 94          942   
                                         985  2017-05-23 95          943   
                                         986  2017-05-23 96          944   
                                         987  2017-05-23 97          945   
                                         988  2017-05-23 98          946   
                                         1139 2017-05-24 100        1097   
                                                         99         1097   
                                         1140 2017-05-24 102        1098   
                                                         101        1098   
                                         1141 2017-05-24 104        1099   
                                                         103        1099   
                                         1142 2017-05-24 106        1100   
                                                         105        1100   
                                         1143 2017-05-24 108        1101   
                                                         107        1101   
                                         1144 2017-05-24 110        1102   
                                                         109        1102   
                                         1145 2017-05-24 112        1103   
                                                         111        1103   
                                         1146 2017-05-24 114        1104   
                                                         113        1104   
                                         1147 2017-05-24 116        1105   
                                                         115        1105   
...                                                                  ...   
                                         1151 2017-05-24 119        1109   
7a2306cbdeb6c8d6087279f0df15bbfdb3df0218 868  2017-05-22 24          826   
                                         869  2017-05-22 27          827   
                                         870  2017-05-22 30          828   
                                         871  2017-05-22 33          829   
                                         872  2017-05-22 36          830   
                                         873  2017-05-22 39          831   
                                         874  2017-05-22 42          832   
                                         875  2017-05-22 45          833   
                                         876  2017-05-22 48          834   
                                         877  2017-05-22 51          835   
                                         888  2017-05-22 54          846   
                                         889  2017-05-22 57          847   
                                         891  2017-05-22 60          849   
                                         1205 2017-05-24 64         1163   
                                                         63         1163   
                                         1206 2017-05-24 67         1164   
                                                         66         1164   
                                         1207 2017-05-24 70         1165   
                                                         69         1165   
                                  

In [44]:
answered_df

Unnamed: 0  \
hash_fin                                 id_x workday                      
07e1b75fe53343221703226deb96b71bb6d90930 985  2017-05-23 95          943   
                                         1142 2017-05-24 106        1100   
                                                         105        1100   
7a2306cbdeb6c8d6087279f0df15bbfdb3df0218 891  2017-05-22 60          849   
                                         1205 2017-05-24 64         1163   
                                                         63         1163   

                                                              id_x  \
hash_fin                                 id_x workday                
07e1b75fe53343221703226deb96b71bb6d90930 985  2017-05-23 95    985   
                                         1142 2017-05-24 106  1142   
                                                         105  1142   
7a2306cbdeb6c8d6087279f0df15bbfdb3df0218 891  2017-05-22 60    891   
                                         1205 2017-05-24 64   1205   
                                                         63   1205   

                                                              checklist_id  \
hash_fin                                 id_x workday                        
07e1b75fe53343221703226deb96b71bb6d90930 985  2017-05-23 95              1   
                                         1142 2017-05-24 106             1   
                                                         105             1   
7a2306cbdeb6c8d6087279f0df15bbfdb3df0218 891  2017-05-22 60              1   
                                         1205 2017-05-24 64              1   
                                                         63              1   

                                                             question_id  \
hash_fin                                 id_x workday                      
07e1b75fe53343221703226deb96b71bb6d90930 985  2017-05-23 95        $0301   
                                         1142 2017-05-24 106       $0301   
                                                         105       $0301   
7a2306cbdeb6c8d6087279f0df15bbfdb3df0218 891  2017-05-22 60        $0301   
                                         1205 2017-05-24 64        $0301   
                                                         63        $0301   

                                                                workday  \
hash_fin                                 id_x workday                     
07e1b75fe53343221703226deb96b71bb6d90930 985  2017-05-23 95  2017-05-23   
                                         1142 2017-05-24 106 2017-05-24   
                                                         105 2017-05-24   
7a2306cbdeb6c8d6087279f0df15bbfdb3df0218 891  2017-05-22 60  2017-05-22   
                                         1205 2017-05-24 64  2017-05-24   
                                                         63  2017-05-24   

                                                             response  \
hash_fin                                 id_x workday                   
07e1b75fe53343221703226deb96b71bb6d90930 985  2017-05-23 95         N   
                                         1142 2017-05-24 106        N   
                                                         105        N   
7a2306cbdeb6c8d6087279f0df15bbfdb3df0218 891  2017-05-22 60         N   
                                         1205 2017-05-24 64         Y   
                                                         63         Y   

                                                                                  date_x  \
hash_fin                                 id_x workday                                      
07e1b75fe53343221703226deb96b71bb6d90930 985  2017-05-23 95   2017-05-23 14:54:32.821528   
                                         1142 2017-05-24 106  2017-05-24 14:43:41.934523   
                                                         105  2017-05-24 14:43:41.934523   
7a2306cbdeb6

# NSICU (unit 3)
01 ventilatorWeaning 

03 linesFood 

06 antibiotics 

07 dvt 

08 pressureUlcer 

10 NO 

### CTICU (unit 4)
01 ventilatorWeaning 

03 linesFood 

06 antibiotics 

07 dvt 

08 pressureUlcer 

10 NO 

In [73]:
checklist_query = "select * from backend_v10_checklist"
df_checklist = pd.read_sql(checklist_query, conn)
df_checklist

,id,public_id,name,text,in_production,unit_id
0,2,20,Test,"{""id"": ""20"", ""groups"": [{""id"": ""01"", ""dependen...",1,1
1,5,1,Initial checklist,"{""order"": ""(01,{03|06|07|08},10)"", ""id"": ""1"", ...",1,3
2,10,1,Initial checklist,"{""order"": ""(01,{03|06|07|08},10)"", ""id"": ""1"", ...",1,4
3,11,4,No ARDS with N/A (lowercase),"{""order"": ""(01,{03|06|07|08},10)"", ""groups"": [...",1,8
4,12,100,Leigh Wild Checklist,"{""order"": ""(01,{03|06|07|08},10)"", ""groups"": [...",1,8
5,13,2,Initial checklist with tracheostomy question,"{""description"": ""Initial checklist with trache...",1,3
6,14,2,Initial checklist with tracheostomy question,"{""description"": ""Initial checklist with trache...",1,4


In [94]:
import ast
unit3_2 =ast.literal_eval(df_checklist.text[2])
unit3_1=ast.literal_eval(df_checklist.text[6])

In [95]:
for key, val in unit3_1.items():
    print(key, val, type(val),len(val))
    print()

id 2 <class 'str'> 1

order (01,{03|06|07|08},10) <class 'str'> 21

groups [{'emrInfoId': 'ventilatorWeaning', 'sections': [{'id': '$0100', 'title': 'Sedation:', 'dependence': 'hasMedication', 'questions': [{'text': 'Has a sedation awakening trial (SAT) been performed today?', 'id': '$0101', 'dependence': 'true', 'indentation': 0, 'questionType': 'Q'}, {'text': 'Is it appropriate to perform an SAT today?', 'id': '$0102', 'dependence': "$0101=='N'", 'indentation': 0, 'questionType': 'A'}]}, {'id': '$0200', 'title': 'Ventilator weaning:', 'dependence': 'hasVentilation', 'questions': [{'text': 'Has a spontaneous breathing trial (SBT) been performed today?', 'id': '$0201', 'dependence': 'true', 'indentation': 0, 'questionType': 'Q'}, {'text': 'Can the patient be extubated today?', 'id': '$0202', 'dependence': "$0201=='Y'", 'indentation': 0, 'questionType': 'Q'}, {'text': 'Does the patient have a tracheostomy or is scheduled for tracheostomy placement?', 'id': '$0205', 'dependence': "$0201=

In [96]:
for a in unit3_1['groups']:
    print(a['id'], a['emrInfoId'], '\n')

01 ventilatorWeaning 

03 linesFood 

06 antibiotics 

07 dvt 

08 pressureUlcer 

10 NO 



In [97]:
for b in unit3_2['groups']:
    print(b['id'], b['emrInfoId'], '\n')

01 ventilatorWeaning 

03 linesFood 

06 antibiotics 

07 dvt 

08 pressureUlcer 

10 NO 

